In [4]:
import numpy as np
import pandas as pd
import math
import itertools
import matplotlib.pyplot as plt
import torch

import import_ipynb
from Environments import Square_Crossroads
from Actor_Critic import Actor_Critic

In [5]:
n_times_steps = 3
seed = 10
dist_cars = 0.5

env = Square_Crossroads(n_times_steps, seed, dist_cars)

print(env.states)
action_space = {0:[0,0], 1:[0,0], 2:[0,0], 3:[0,0]}
# total_reward = 0

state_tensor = torch.zeros([4,2], dtype=torch.float32)


ac = Actor_Critic(state_tensor, 20, 400, learning_rate=3e-4)

Environment reset with param
{0: [[0, 5], [5, 10], 0, 0], 1: [[5, 0], [5, 10], 0, 0], 2: [[5, 10], [0, 5], 0, 0], 3: [[10, 5], [0, 5], 0, 0]}


In [6]:
ac

Actor_Critic(
  (critic_linear1): Linear(in_features=2, out_features=400, bias=True)
  (critic_linear2): Linear(in_features=400, out_features=1, bias=True)
  (actor_linear1): Linear(in_features=2, out_features=20, bias=True)
  (actor_linear2): Linear(in_features=20, out_features=20, bias=True)
  (alpha): Linear(in_features=20, out_features=1, bias=True)
  (beta): Linear(in_features=20, out_features=1, bias=True)
)